In [1]:
import selenium

from selenium import webdriver 
from selenium.webdriver import ActionChains 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time

import pygsheets

from datetime import timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd 
import math
from datetime import datetime 

from bs4 import BeautifulSoup
import numpy as np
import re

from time import sleep
import json
import matplotlib.pyplot as plt

import urllib
from urllib import request
import re

import schedule

import urllib.request
from datetime import datetime, timedelta
import pandas as pd
import json


from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *

In [2]:
BASE_URL = 'https://api.naver.com'
API_KEY = '****************'
SECRET_KEY = '*********'
CUSTOMER_ID = '*****'

In [3]:
rel = RelKwdStat(BASE_URL, API_KEY, SECRET_KEY, CUSTOMER_ID)

#### powernad 모듈 임포트

In [4]:
def search_keyword(searchword):
    

    kwdDataList = rel.get_rel_kwd_stat_list(siteId=None, biztpId=None, hintKeywords=searchword, event=None, month=None, showDetail='1')


    kwd_result = (kwdDataList[0].relKeyword, #키워드
                 kwdDataList[0].monthlyPcQcCnt, #월간 검색수 (PC)
                 kwdDataList[0].monthlyMobileQcCnt, # 월간 검색수 (Mobile)
                 kwdDataList[0].monthlyPcQcCnt+kwdDataList[0].monthlyMobileQcCnt) # 월간 total 

    return(kwd_result[3])

#### 구글 스프레드 시트 API 
####   L pygsheets 모듈 사용

In [5]:
gc = pygsheets.authorize(outh_file = 'secret.json')

In [6]:
#시트 불러오기 

file_nm = '구글스프레드시트 이름'

sh = gc.open(file_nm) 
sheet1 =sh.sheet1 

In [7]:
#2번째 이상 시트 불러오기

sheet_list = sh.worksheets() 

sheet2 = sh.worksheet_by_title('시트이름')


### 무한루프 
#### L 네이버 트랜드 지수 API 연동 후 네이버 트랜드 지수의 비중 * 네이버 광고 최근 한달 총 검색량 
#### L 구글 스프레드 시트에 Update

In [8]:


def job():
    
    #### 키워드

    keyword_list = ['원하는 키워드 입력']

    
    #### 연관 검색어 리스트 
    
    temp = [] 

    for i in range(len(keyword_list)):
    
        sleep(0.1)

        try:
            URL = "https://search.naver.com/search.naver?ie=UTF-8&sm=whl_hty&query={}".format(keyword_list[i])
            chrome_options = Options()
            driver = webdriver.Chrome(executable_path='chromedriver',chrome_options=chrome_options)
            driver.implicitly_wait(3)
            driver.get(url=URL)

            driver.implicitly_wait(3)

            searches = driver.find_elements_by_css_selector(".lst_related_srch li")



            for keyword in searches:
                result = keyword.text
                temp.append(result)



            driver.close()


        except NameError:
            print(i)
            
    
    ### 유사 검색어 리스트 
    tempt=[]
    
    
    for tmp in range(len(keyword_list)):
        sleep(0.1)
        URL = 'https://www.naver.com'
        chrome_options = Options()
        driver = webdriver.Chrome(executable_path='chromedriver',chrome_options=chrome_options)
        driver.implicitly_wait(3)
        driver.get(url=URL)
        key = keyword_list[tmp]
        driver.find_element_by_name('query').send_keys(key)
        driver.implicitly_wait(3)

        class_names = driver.find_elements_by_css_selector('#autoFrame > div > div > div.atcmp_fixer._atcmp_layer > div.atcmp_container._words > ul>li')


        for i in class_names:
            if ((i.text != key) and(i.text is not None)) : 
                class_name = i.text

                nn= class_name.replace('\n추가',"")
                tempt.append(nn)
        sleep(0.1) 
        driver.close()

        
        
    
    # new 키워드 리스트 파악 
    
    new_keyword = temp+ tempt
    
    #기존 검색 키워드 제거
    final= [] 

    for j in new_keyword : 
        if j not in keyword_list : 
            final.append(j)
            
            
    # 기존 단어는 뒤로 붙이기
    
    final = keyword_list + final 
    
    
    
    
    
    ### 네이버 데이터랩 검색어 트랜드 API
    
    
    today = str(datetime.now().date())

    # api token
    client_id = "******" 
    client_secret = "********"
    
    
    
    # 날짜 

    toda = datetime.now()
    time_month = toda - relativedelta(months=1)
    time_month = time_month.strftime('%Y-%m-%d')
    time_month= str(time_month)
    time_month

    today = str(datetime.now().date())
    
    
    #빈값 제거 
    final = [v for v in final if v ]
    final =[v.replace(" ","") for v in final ]
    
    
    
    # 중복값 제거 

    new_list = []

    for i in final: 
        if i not in new_list : 
            new_list.append(i)

            
            
    

    
    ### 리스트 나눠서 진행
    
    
    num =math.floor(len(new_list) /3)
    
    
    final_1=new_list[:num]
    final_2=new_list[num:num*2]
    final_3 = new_list[num*2:]
    
    error=[]
    
    
    ### 구글 스프레드 시트 연동 및 네이버 한달 광고 연동############
    
    for i in final_1:  
        sleep(1)
        try:
            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]

            worldvision = [a['ratio'] for a in result['results'][0]['data']]

            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))


        # 구글 스프레드 시트 연동 


            sheet1.append_table([today,aa.iloc[-1,:].values.tolist()[0],i,aa.iloc[-1,:].values.tolist()[1]])

        except (TypeError, IndexError):
            print(" 타입 or 인덱스 에러:",i)
            error.append(i)

        except HTTPError:
            print('http 에러:', i )
            error.append(i)
            
    
    
    
    sleep(3)
    
    
                
    for i in final_2:  
        sleep(1)
        try:
            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]

            worldvision = [a['ratio'] for a in result['results'][0]['data']]

            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))


        # 구글 스프레드 시트 연동 


            sheet1.append_table([today,aa.iloc[-1,:].values.tolist()[0],i,aa.iloc[-1,:].values.tolist()[1]])

        except (TypeError, IndexError):
            print(" 타입 or 인덱스 에러:",i)
            error.append(i)
        except HTTPError:
            print('http 에러:', i )
            error.append(i)
            
    sleep(3)
    
    
    
    for i in final_3:  
        sleep(1)
        try:
            url = "https://openapi.naver.com/v1/datalab/search"
            body = "{\"startDate\":\""+time_month+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\""+i+"\",\"keywords\":[\""+i+"\"]}]}";
            requested = urllib.request.Request(url)
            requested.add_header("X-Naver-Client-Id", client_id)
            requested.add_header("X-Naver-Client-Secret", client_secret)
            requested.add_header("Content-Type", "application/json")
            response = urllib.request.urlopen(requested, data=body.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                output_data = response_body.decode('utf-8')
            else:
                print('Error code:'+ rescode)
                pass

            result = json.loads(output_data)


            date = [a['period'] for a in result['results'][0]['data']]

            worldvision = [a['ratio'] for a in result['results'][0]['data']]

            aa = pd.DataFrame({'date':date, 
                          i:[a['ratio'] for a in result['results'][0]['data']],
                          })

            # 일일 데이터 계산 


            sleep(0.5)
            total = aa[i].sum()


            searchword = i.replace(" ","")
            a = search_keyword(searchword)


            aa[i] = aa[i].apply(lambda x :((x / total)*float(a)))


        # 구글 스프레드 시트 연동 


            sheet1.append_table([today,aa.iloc[-1,:].values.tolist()[0],i,aa.iloc[-1,:].values.tolist()[1]])

        except (TypeError, IndexError):
            print(" 타입 or 인덱스 에러:",i)
            error.append(i)
        except HTTPError:
            print('http 에러:', i )
            error.append(i)
            
    sleep(3)
  



    ### sheet1 데이터 갖고오기 
    
    df = sheet1.get_as_df()
    
    
    ### sheet2 검수 페이지 업데이트 
    sheet2.append_table([today,len(new_list),len(error),len(df[df['수집날짜']==today])])

#### 스케쥴러로 매일 08시34분 마다 실행

In [ ]:
schedule.every().day.at("08:34").do(job)

while True:
    schedule.run_pending()
    
    time.sleep(1)

<ipython-input-8-79b6a9cea110>:23: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='chromedriver',chrome_options=chrome_options)
<ipython-input-8-79b6a9cea110>:54: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='chromedriver',chrome_options=chrome_options)


 타입 or 인덱스 에러: 유니셰프팔찌
 타입 or 인덱스 에러: 굿네이버스장려상
 타입 or 인덱스 에러: 굿네이버스본부장상
 타입 or 인덱스 에러: 굿네이버스상장편지
 타입 or 인덱스 에러: 굿네이버스상장희망
 타입 or 인덱스 에러: 굿네이버스상장대회
 타입 or 인덱스 에러: 굿네이버스상장지역
 타입 or 인덱스 에러: 굿네이버스상장쓰기
 타입 or 인덱스 에러: 월드비전서울동부지역본부
 타입 or 인덱스 에러: 초록우산어린이재단인천지역본부(미추홀구)
